In [2]:
import pandas as pd
df = pd.read_csv("/Users/iyeeun/Documents/coreview_3/data/df_sumWordSentence.csv")
df.head()

,이름,작성일자,평점,리뷰 내용,맛 만족도,preprocessed_review,krwordrank
0,박*혜,2025.01.14,5,"짜파게티 마라, 어떤 제품일까?먼저 짜파게티 마라의 기본 정보를 알아보았어요. 이 ...",아주 넉넉해요,짜파게티 마라 어떤 제품 일까먼저 짜파게티 마라 기본 정보를 알아보았어요 제품 유명...,['마라']
1,dimss,2025.01.13,4,안녕하세요!! 리뷰하는 dimss입니다^^오늘도 솔직 사용후기 들고 왔어요❤️⬇️⬇...,적당히 남았어요,안녕하세요 리뷰 하는 입니다 오늘도 솔직 사용후기 들고 왔어요짜파게티 마라 솔직 후...,"['마라', '분들', '짜파']"
2,이*람,2025.01.12,5,### 구매 계기매운 음식을 좋아하는 저는 최근 마라탕이나 마라샹궈와 같은 마라 요...,적당히 남았어요,구매 계기매운 음식 좋아 하는 저는 최근 마라 탕이나 마라 샹궈와 같은 마라 요리 ...,"['마라', '매운', '맛이', '정말', '국물']"
3,시마에나가,2025.01.11,4,"⏹️ 요약 ⏹️✅ 구성 - 면, 후레이크, 짜장스프, 마라조미유✅ 맛 - 맵찔이 먹...",적당히 남았어요,요약 구성 후레이크 짜장스프 마라 조미유 맵찔이 먹으면 안됨 그냥 매운 마라 구매 ...,"['마라', '넣고', '짜파', '먹어']"
4,김*수,2025.01.11,5,짜파게티 마라를 구매한 후기를 이모티콘을 활용하여 작성해 보겠습니다. 아래와 같은 ...,아주 넉넉해요,짜파게티 마라 구매 후기 이모티콘을 활용 하여 작성해 보겠습니다 아래와 같은 방식 ...,"['마라', '매운', '맛이']"


In [10]:
import pandas as pd

def analyze_keyword_ratings(df, keyword_column='krwordrank', rating_column='평점'):
    # Convert strings in the keyword column to lists, if necessary
    df[keyword_column] = df[keyword_column].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Flatten all keywords into a single list
    all_keywords = [keyword for keywords_list in df[keyword_column] if isinstance(keywords_list, list) for keyword in keywords_list]

    # Count the occurrences of each keyword
    keyword_counts = pd.Series(all_keywords).value_counts().reset_index()
    keyword_counts.columns = ['Keyword', 'Count']

    # Calculate the average rating for each keyword
    keyword_avg_ratings = []
    for keyword in keyword_counts['Keyword']:
        # Filter rows containing the current keyword
        filtered_df = df[df[keyword_column].apply(lambda x: keyword in x if isinstance(x, list) else False)]
        # Calculate the average rating for these rows
        avg_rating = filtered_df[rating_column].mean()
        keyword_avg_ratings.append(avg_rating)

    # Add the average ratings to the keyword counts DataFrame
    keyword_counts['Average_Rating'] = keyword_avg_ratings

    return keyword_counts

# Example usage
result = analyze_keyword_ratings(df)
result[:5]

,Keyword,Count,Average_Rating
0,마라,55,4.836364
1,짜파,18,4.611111
2,짜파게티,13,4.923077
3,매운,11,4.909091
4,맛이,10,5.000000


In [17]:
import pandas as pd
import re

def analyze_keyword_reviews(data, keyword, review_column='preprocessed_review'):
    # Function to extract and summarize sentences containing the keyword
    def summarize_keyword_reviews(text):
        if pd.isna(text):
            return None
        sentences = re.split(r'[.!?]', text)  # Split into sentences
        keyword_sentences = [s.strip() for s in sentences if keyword in s]  # Filter sentences containing the keyword
        return keyword_sentences

    # Function to count occurrences of the keyword in a text
    def count_keyword_mentions(text):
        if pd.isna(text):
            return 0
        return text.count(keyword)

    # Extract keyword-related sentences
    data['keyword_sentences'] = data[review_column].apply(summarize_keyword_reviews)

    # Count keyword mentions in each review
    data['keyword_count'] = data[review_column].apply(count_keyword_mentions)

    # Sort the data by the number of keyword mentions, descending
    top_keyword_reviews = data.sort_values(by='keyword_count', ascending=False).head(5)

    # Reset index to ensure numerical order
    top_keyword_reviews = top_keyword_reviews.reset_index(drop=True)

    # Select relevant columns for output
    top_reviews = top_keyword_reviews[[review_column, 'keyword_count']]

    return top_reviews

# Example usage
result = analyze_keyword_reviews(df, keyword='마라')
result


,preprocessed_review,keyword_count
0,짜파게티 마라 내돈내산으로 솔직 후기 입니다평소에 마라 탕을 엄청 좋아 하는 사람 ...,24
1,구매 계기매운 음식 좋아 하는 저는 최근 마라 탕이나 마라 샹궈와 같은 마라 요리 ...,20
2,짜파게티 마라 새로 나와서 너무 궁금 해서 주문 해봤습니다먹어본 소감 부터 적어보겠...,18
3,다양한 마라 제품이 판을 치고 있는 시점에서나올듯 나오 않을 듯하던 제품 드디어 나...,17
4,제품 명짜파게티 마라 개마라짜파게티 나왔다 길래 냉큼 시켜 봤습니다 요즘 뭐만하면 ...,17


In [63]:
import pandas as pd
import re
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load tokenizer and model for summarization
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def analyze_and_summarize_reviews(data, keyword, review_column='preprocessed_review', max_length=50, min_length=50):
    """
    Analyzes reviews to extract keyword-related insights and summarizes them.

    Parameters:
        data (pd.DataFrame): Input DataFrame containing review data.
        keyword (str): The keyword to analyze and summarize.
        review_column (str): Column name containing review texts.
        max_length (int): Maximum length of the summary.
        min_length (int): Minimum length of the summary.

    Returns:
        pd.DataFrame: A DataFrame containing columns:
                      - preprocessed_review: Original review text.
                      - keyword_count: Number of times the keyword appears in the review.
                      - summary: Summarized text of the review.
    """
    # Function to extract and summarize sentences containing the keyword
    def summarize_keyword_reviews(text):
        if pd.isna(text):
            return None
        sentences = re.split(r'[.!?]', text)  # Split into sentences
        keyword_sentences = [s.strip() for s in sentences if keyword in s]  # Filter sentences containing the keyword
        return keyword_sentences

    # Function to count occurrences of the keyword in a text
    def count_keyword_mentions(text):
        if pd.isna(text):
            return 0
        return text.count(keyword)

    # Extract keyword-related sentences
    data['keyword_sentences'] = data[review_column].apply(summarize_keyword_reviews)

    # Count keyword mentions in each review
    data['keyword_count'] = data[review_column].apply(count_keyword_mentions)

    # Sort the data by the number of keyword mentions, descending
    top_keyword_reviews = data.sort_values(by='keyword_count', ascending=False).head(5)

    # Reset index to ensure numerical order
    top_keyword_reviews = top_keyword_reviews.reset_index(drop=True)

    # Summarize reviews using KoBART
    def summarize(text):
        if pd.isna(text) or len(text.strip()) == 0:
            return None
        inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(
            inputs,
            max_length=max_length,
            min_length=min_length,
            length_penalty=2.0,
            num_beams=4,
            repetition_penalty=2.0,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Ensure the summary starts with the keyword
        if not summary.startswith(keyword):
            sentences = summary.split(".")
            keyword_sentence = next((s for s in sentences if keyword in s), None)
            if keyword_sentence:
                summary = f"{keyword} {keyword_sentence.strip()}"

        return summary[:100]  # Limit to 100 characters

    top_keyword_reviews['summary'] = top_keyword_reviews[review_column].apply(summarize)

    # Select relevant columns for output
    result = top_keyword_reviews[[review_column, 'keyword_count', 'summary']]

    return result

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


                                 preprocessed_review  keyword_count  \
0  짜파게티 마라 내돈내산으로 솔직 후기 입니다평소에 마라 탕을 엄청 좋아 하는 사람 ...             24   
1  구매 계기매운 음식 좋아 하는 저는 최근 마라 탕이나 마라 샹궈와 같은 마라 요리 ...             20   
2  짜파게티 마라 새로 나와서 너무 궁금 해서 주문 해봤습니다먹어본 소감 부터 적어보겠...             18   
3  다양한 마라 제품이 판을 치고 있는 시점에서나올듯 나오 않을 듯하던 제품 드디어 나...             17   
4  제품 명짜파게티 마라 개마라짜파게티 나왔다 길래 냉큼 시켜 봤습니다 요즘 뭐만하면 ...             17   

                                             summary  
0  마라 짜파게티와 마라 환상적인 조화 고소한 짜파게티 얼얼 마라 더해 환상의 맛조합화...  
1  마라 짜파게티 본래의 짜장 맛이 좋아 자주 먹곤 했는데 마라 맛까지 추가되어 여러 ...  
2  마라 짜파게티의 보다 마라 맛이 지배적이어서 짜파게티 느낌이 아니라 마라 맛이 지배...  
3  마라 짜파게티 포장지 색상이 보라색인지 모르겠 지만 일단 눈에 들어 오긴 하며 여러...  
4  마라 맵짜파게티 개마라짜파게티 나왔다 길래 냉큼 시켜 봤는데 마라 소스 넣자마자 마...  


In [74]:
result['summary']

0    마라 짜파게티와 마라 환상적인 조화 고소한 짜파게티 얼얼 마라 더해 환상의 맛조합화...
1    마라 짜파게티 본래의 짜장 맛이 좋아 자주 먹곤 했는데 마라 맛까지 추가되어 여러 ...
2    마라 짜파게티의 보다 마라 맛이 지배적이어서 짜파게티 느낌이 아니라 마라 맛이 지배...
3    마라 짜파게티 포장지 색상이 보라색인지 모르겠 지만 일단 눈에 들어 오긴 하며 여러...
4    마라 맵짜파게티 개마라짜파게티 나왔다 길래 냉큼 시켜 봤는데 마라 소스 넣자마자 마...
Name: summary, dtype: object